In [1]:
import logging
import sys
import warnings
from urllib.parse import urlparse

import numpy as np
import pandas as pd
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature

logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)


def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

warnings.filterwarnings("ignore")
np.random.seed(40)

# Read the wine-quality csv file from the URL
csv_url = (
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-red.csv"
)
try:
    data = pd.read_csv(csv_url, sep=";")
except Exception as e:
    logger.exception(
        "Unable to download training & test CSV, check your internet connection. Error: %s", e
    )

# Split the data into training and test sets. (0.75, 0.25) split.
train, test = train_test_split(data)

# The predicted column is "quality" which is a scalar from [3, 9]
train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y = test[["quality"]]

alpha = 0.5
l1_ratio = 0.5

In [2]:
with mlflow.start_run():
    lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
    lr.fit(train_x, train_y)

    predicted_qualities = lr.predict(test_x)

    (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

    print(f"Elasticnet model (alpha={alpha:f}, l2_ratio={l1_ratio:f}):")
    print(f"  RMSE: {rmse}")
    print(f"  MAE: {mae}")
    print(f"  R2: {r2}")

    mlflow.log_param("alpha", alpha)
    mlflow.log_param("l1_ratio", l1_ratio)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)

    predictions = lr.predict(train_x)
    signature = infer_signature(train_x, predictions)

    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

    # Model registry does not work with file store
    if tracking_url_type_store != "file":
        # Register the model
        # There are other ways to use the Model Registry, which depends on the use case,
        # please refer to the doc for more information:
        # https://mlflow.org/docs/latest/model-registry.html#api-workflow
        mlflow.sklearn.log_model(
            lr, "model", registered_model_name="ElasticnetWineModel", signature=signature
        )
    else:
        mlflow.sklearn.log_model(lr, "model", signature=signature)

Elasticnet model (alpha=0.500000, l2_ratio=0.500000):
  RMSE: 0.7931640229276851
  MAE: 0.6271946374319586
  R2: 0.10862644997792614


In [3]:
signature

inputs: 
  ['fixed acidity': double, 'volatile acidity': double, 'citric acid': double, 'residual sugar': double, 'chlorides': double, 'free sulfur dioxide': double, 'total sulfur dioxide': double, 'density': double, 'pH': double, 'sulphates': double, 'alcohol': double]
outputs: 
  [Tensor('float64', (-1,))]
params: 
  None

In [4]:
tracking_url_type_store

'file'

In [5]:
!mlflow ui --port=5001

[2023-09-27 20:54:45 -0400] [342474] [INFO] Starting gunicorn 21.2.0
[2023-09-27 20:54:45 -0400] [342474] [INFO] Listening at: http://127.0.0.1:5001 (342474)
[2023-09-27 20:54:45 -0400] [342474] [INFO] Using worker: sync
[2023-09-27 20:54:45 -0400] [342475] [INFO] Booting worker with pid: 342475
[2023-09-27 20:54:45 -0400] [342476] [INFO] Booting worker with pid: 342476
[2023-09-27 20:54:45 -0400] [342477] [INFO] Booting worker with pid: 342477
[2023-09-27 20:54:45 -0400] [342478] [INFO] Booting worker with pid: 342478
^C
[2023-09-27 20:56:11 -0400] [342474] [INFO] Handling signal: int
[2023-09-27 20:56:11 -0400] [342478] [INFO] Worker exiting (pid: 342478)
[2023-09-27 20:56:11 -0400] [342477] [INFO] Worker exiting (pid: 342477)
[2023-09-27 20:56:11 -0400] [342475] [INFO] Worker exiting (pid: 342475)
[2023-09-27 20:56:11 -0400] [342476] [INFO] Worker exiting (pid: 342476)
